In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# Download the IMDB dataset
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=10000)

# Pad sequences to ensure uniform length
max_length = 256
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, maxlen=max_length)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=max_length)

# Create a simple model with an embedding layer instead of using Hub directly
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data, train_labels,
    epochs=5,  # Using fewer epochs for quicker execution
    batch_size=512,
    validation_split=0.2
)

# Evaluate the model
results = model.evaluate(test_data, test_labels)
print(f"Test accuracy: {results[1]:.3f}")

# Make predictions on a few examples
predictions = model.predict(test_data[:5])
print("Predictions:", predictions.flatten())
print("Actual labels:", test_labels[:5])


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.5007 - loss: 0.6909 - val_accuracy: 0.5062 - val_loss: 0.6773
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.4979 - loss: 0.6711 - val_accuracy: 0.5068 - val_loss: 0.6464
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.5127 - loss: 0.6366 - val_accuracy: 0.5852 - val_loss: 0.5969
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.6026 - loss: 0.5815 - val_accuracy: 0.7528 - val_loss: 0.5392
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7263 - loss: 0.5146 - val_accuracy: 0.7438 - val_loss: 0.4776
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7388 - loss: 0.4814
Test accuracy: 0.733
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predictions: [-0.39121068  1.6034985   0.00801477 -0.17339984  0.23511323]
Actual labels: [0 1 1 0 1]


In [5]:
# Function to predict sentiment of new text
def predict_sentiment(text, word_index):
    # Convert text to sequence of integers
    words = text.lower().split()
    sequence = [word_index.get(word, 0) for word in words]
    # Pad sequence to required length
    padded = tf.keras.preprocessing.sequence.pad_sequences([sequence], maxlen=max_length)
    # Get prediction
    prediction = model.predict(padded)[0][0]
    # Flip the interpretation if needed
    return {
        'text': text,
        'sentiment': 'Negative' if prediction < 0 else 'Positive',
        'confidence': float(abs(prediction))
    }


# Get the word index for encoding new text
word_index = tf.keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

# Test with some example reviews
example_reviews = [
    "This movie was excellent! I loved every minute of it.",
    "I hated this movie. The plot was terrible and the acting was worse.",
    "The film was okay, but I've seen better."
]

for review in example_reviews:
    result = predict_sentiment(review, word_index)
    print(f"Text: {result['text']}")
    print(f"Sentiment: {result['sentiment']} (confidence: {result['confidence']:.2f})")
    print("-" * 50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Text: This movie was excellent! I loved every minute of it.
Sentiment: Positive (confidence: 0.05)
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Text: I hated this movie. The plot was terrible and the acting was worse.
Sentiment: Negative (confidence: 0.20)
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Text: The film was okay, but I've seen better.
Sentiment: Positive (confidence: 0.09)
--------------------------------------------------
